# General preparations

In [5]:
from misc.misc import run_in_thread_simple, run_in_thread
%load_ext autoreload
%autoreload 2
# %matplotlib notebook
import matplotlib
matplotlib.use('TKAgg')
import os 
import datetime
import glob 
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_modes2 import gen_input_spots_array, gen_output_modes_Unitary
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan, get_signal_scanner, get_idler_scanner
from pianoq.lab.mplc.mask_utils import remove_input_modes
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.misc.misc import run_in_thread, run_in_thread_simple
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_QKD_masks, create_WFM_unitary_masks
import time 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_10_09_fixing_phases_different_Us'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

In [3]:
# MPLC
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

Got MPLC


In [4]:
def get_masks(U_no):
    paths = glob.glob(fr'{dir_path}\U{U_no}U*.masks')
    assert len(paths) == 1, 'should be only one '
    msks = MPLCMasks()
    msks.loadfrom(paths[0])
    return msks.real_masks
    
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

## Checklist
- [X] make list
- [X] create masks WFM
- [X] measure single counts 
- [X] mark spots locations 
- [ ] find phases
- [ ] measure coincidence no phases
- [ ] measure with good phases 
- [ ] measure with random phases X 5 

# Create masks WFM

In [6]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_10_09_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
mplc_sims = []
for U_no in range(5):
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    masks_path = fr'{dir_path}\U{U_no}U_{timestamp}.masks'
    mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path)
    mplc_sims.append(mplc_sim)

100%|██████████| 15/15 [02:46<00:00, 11.10s/it]


In [ ]:
for U_no in range(5):
    mplc_sims[U_no].res._calc_normalized_overlap()
    display(np.angle(np.diag(mplc_sims[U_no].res.forward_overlap)))
    display(np.diag(mplc_sims[U_no].res.forward_overlap))
    mplc_sims[U_no].res._calc_fidelity()
    display(mplc_sims[U_no].res.fidelity)

In [8]:
for U_no in range(5):
    mplc_sims[U_no].res.show_overlap()

In [12]:
for U_no in range(5):
    fig, axes = plt.subplots(2, 5)
    for i, ax in enumerate(axes.flat):
        # ax.imshow(np.angle(res.masks[i][360:720, 140:280]), cmap='gray')
        ax.imshow(np.angle(mplc_sims[U_no].res.masks[i][360:720, 140:280]), cmap='gray')
        fig.show()

In [15]:
mplc_sims[3].res.show_all(4)

# Find locations with single counts 

In [5]:
resolution = 4

scanner_sig, x_motor_sig, y_motor_sig, time_tagger_sig = get_signal_scanner(integration_time=1.0, coin_window=2e-9, resolution=resolution, half_scan=True)
scanner_idl, x_motor_idl, y_motor_idl, time_tagger_idl = get_idler_scanner(integration_time=1.0, coin_window=2e-9, resolution=resolution, half_scan=True)

for U_no in range(2):
    masks = get_masks(U_no)
    masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
    mplc.load_masks(masks, linear_tilts=True)
    mplc.restore_location()
    
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    singles_idler_path = fr'{dir_path}\{timestamp}_QKD_MUB2_signal_res_{resolution}.scan'
    singles_signal_path = fr'{dir_path}\{timestamp}_QKD_MUB2_idler_res_{resolution}.scan'
    
    scanner_sig.saveto_path = singles_signal_path
    scanner_idl.saveto_path = singles_idler_path
    t = run_in_thread(scanner_sig.scan, x_motor=x_motor_sig, y_motor=y_motor_sig, ph=time_tagger_sig)
    t2 = run_in_thread(scanner_idl.scan, x_motor=x_motor_idl, y_motor=y_motor_idl, ph=time_tagger_idl)    
    t.join()
    t2.join()

## mark spots locations  

In [14]:
from pianoq_results.scan_result import ScanResult

path_sig = glob.glob(fr'{dir_path}\*signal_res_1.scan')[0]
path_idl = glob.glob(fr'{dir_path}\*idler_res_1.scan')[0]

res_sig = ScanResult()
res_idl = ScanResult()
res_idl.loadfrom(path_idl)
res_sig.loadfrom(path_sig)
res_idl.show_singles()
res_sig.show_singles()

In [ ]:
_ = res_sig.get_xys(1, saveto_dir=dir_path)
_ = res_idl.get_xys(2, saveto_dir=dir_path)

# Find phases

In [7]:
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.find_discreet_phases import PhaseFinder
import glob 
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan
import numpy as np
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
import time 


locs_sig_path = glob.glob(fr'{dir_path}\*sig.locs')[0]
locs_idl_path = glob.glob(fr'{dir_path}\*idl.locs')[0]
locs_sig = np.load(locs_sig_path)['locs']
locs_idl = np.load(locs_idl_path)['locs']

masks = orig_masks.copy()
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)

mplc.load_masks(masks, linear_tilts=True)
initiail_phases = np.zeros(25 * 2)

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
phases_path = fr'{dir_path}\{timestamp}_QKD_MUB2_signal.phases'

# i,j of corr matrix that is supposed to be strong 
i = 0
j = 0
pf = PhaseFinder(mplc=mplc, integration_time=30, remote_tagger=True, saveto_path=phases_path,
                 modes_to_keep=modes_to_keep, intial_phases=initiail_phases, coin_window=2e-9)
pf.m_idl_x.move_absolute(locs_idl[i, 0])
pf.m_idl_y.move_absolute(locs_idl[i, 1])
pf.m_sig_x.move_absolute(locs_sig[j, 0])
pf.m_sig_y.move_absolute(locs_sig[j, 1])

Got Zaber motors!
Got Thorlabs motors!
Got TimeTagger!


In [9]:
initiail_phases = np.zeros(25 * 2)
pf.res.phases = initiail_phases
pf.res.initial_phases = initiail_phases

mplc.restore_location()

In [ ]:
pf.find_phases()

In [6]:
pf.close()

In [ ]:
pf.res.phases

In [ ]:
pf.res.plot_best_phases()

3


# Scan correlations 

In [ ]:
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.discrete_photon_scanner import DiscretePhotonScanner
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
from pianoq.lab.mplc.mplc_device import MPLCDevice
import glob
import time 

phases_result = PhaseFinderResult()
phases_path = glob.glob(fr'{dir_path}\*.phases')[0]
phases_result.loadfrom(phases_path)

locs_sig_path = glob.glob(fr'{dir_path}\*sig.locs')[0]
locs_idl_path = glob.glob(fr'{dir_path}\*idl.locs')[0]
locs_sig = np.load(locs_sig_path)['locs']
locs_idl = np.load(locs_idl_path)['locs']


backlash = 0.0
wait_after_move = 0.3
integration_time = 50
coin_window = 0.4e-9
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
phases_type = 'XXX'
scan_path = fr'{dir_path}\{timestamp}_QKD_MUB2_{phases_type}_exp_{integration_time}s.dscan'
dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, coin_window=coin_window)
time.sleep(3)

if True:
    phases_type = 'good'
    phases = phases_result.phases
    
    masks = orig_masks.copy()
    masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
    masks = add_phase_input_spots(masks, phases)
    
    mplc.load_masks(masks, linear_tilts=True)
    mplc.restore_location()
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    used_phases_path = fr'{dir_path}\{timestamp}_used_phases_{phases_type}'
    np.savez(used_phases_path, phase=phases)
    scan_path = fr'{dir_path}\{timestamp}_QKD_MUB2_{phases_type}_exp_{integration_time}s_coin_{int(coin_window*1e12)}.dscan'
    dps.res.path = scan_path
    dps.scan()

if True:
    phases_type = 'none'
    phases = np.zeros_like(phases_result.phases)
    
    masks = orig_masks.copy()
    masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
    masks = add_phase_input_spots(masks, phases)
    
    mplc.load_masks(masks, linear_tilts=True)
    mplc.restore_location()
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    used_phases_path = fr'{dir_path}\{timestamp}_used_phases_{phases_type}'
    np.savez(used_phases_path, phase=phases)
    scan_path = fr'{dir_path}\{timestamp}_QKD_MUB2_{phases_type}_exp_{integration_time}s_coin_{int(coin_window*1e12)}.dscan'
    dps.res.path = scan_path
    dps.scan()

if True:
    phases_type = 'random'
    phases = np.random.uniform(0, 2*np.pi, len(phases_result.phases))
    
    
    for i in range(10):
        # several different random phases 
        masks = orig_masks.copy()
        masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
        # phases = np.random.uniform(0, 2*np.pi, len(phases_result.phases))  # Always random right now <-----------------------------------
        masks = add_phase_input_spots(masks, phases)
        
        mplc.load_masks(masks, linear_tilts=True)
        mplc.restore_location()
        timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
        used_phases_path = fr'{dir_path}\{timestamp}_used_phases_{phases_type}'
        np.savez(used_phases_path, phase=phases)
        scan_path = fr'{dir_path}\{timestamp}_QKD_MUB2_{phases_type}_exp_{integration_time}s_coin_{int(coin_window*1e12)}.dscan'
        dps.res.path = scan_path
        dps.scan()

Got Zaber motors!
Got Thorlabs motors!
Got TimeTagger!
beginning scan
0, 0: 5970.12, 6339.22, 2.42
0, 1: 5970.44, 5779.40, 0.12
0, 2: 5994.76, 5841.92, 0.10


In [6]:
used_phases_path = fr'{dir_path}\{timestamp}_used_phases_{phases_type}'
np.savez(used_phases_path, phase=phases)
dps.scan()

beginning scan
0, 0: 6410.28, 5122.56, 1.94
0, 1: 6410.62, 4742.22, 0.58
0, 2: 6388.90, 4903.08, 0.44
0, 3: 6405.98, 5998.72, 1.28
0, 4: 6392.06, 5643.90, 0.84
1, 0: 7697.28, 5141.18, 0.90
1, 1: 7697.64, 4741.96, 2.28
1, 2: 7690.76, 4937.38, 0.50
1, 3: 7705.68, 6005.56, 0.94
1, 4: 7705.68, 5643.56, 1.16
2, 0: 7531.06, 5150.94, 1.28
2, 1: 7507.12, 4737.52, 0.72
2, 2: 7511.82, 4895.14, 2.80
2, 3: 7490.34, 6002.22, 0.84
2, 4: 7485.66, 5637.12, 1.10
3, 0: 7982.22, 5128.68, 0.82
3, 1: 7998.08, 4729.84, 1.60
3, 2: 7965.64, 4891.74, 0.94
3, 3: 8005.06, 5982.88, 3.00
3, 4: 7974.58, 5612.42, 1.24
4, 0: 7168.86, 5125.40, 0.42
4, 1: 7151.06, 4710.30, 0.30
4, 2: 7147.98, 4896.40, 1.18
4, 3: 7160.88, 5998.80, 1.32
4, 4: 7182.10, 5642.08, 2.36


In [7]:
dps.close()

In [10]:
dps.res.show()
dps.res.show_singles()

In [ ]:
mplc.close()